In [4]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib notebook

import serial
import numpy as np
import matplotlib.pyplot as plt
import time
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from scipy.io import wavfile
from difflib import SequenceMatcher
%matplotlib notebook

In [5]:
"read all 40 questions in qbank"
with open ("Book3.csv", "r") as f:
    data = f.read()
#data.split("\n")
qbank = []

for line in data.split("\n"):
    if ',' not in line:
        continue
    q,a = line.strip().split(',')
    if a[0].lower() == "t":
        qbank.append((q,True))
    else:
        qbank.append((q,False))

# for line in qbank:
#     print(line)

In [6]:
"read all 40 questions and split into 4 qbanks (each has 10 questions)"
with open ("Book3.csv", "r") as f:
    data = f.read()
#data.split("\n")
qbank1 = []
qbank2 = []
qbank3 = []
qbank4 = []
for line in data.split("\n"):
    if ',' not in line:
        continue
    q,a = line.strip().split(',')
    if len(qbank1) <10 :
        if a[0].lower() == "t":
            qbank1.append((q,True))
        else:
            qbank1.append((q,False))
    elif len(qbank2) <10 :
        if a[0].lower() == "t":
            qbank2.append((q,True))
        else:
            qbank2.append((q,False))
    elif len(qbank3) <10 :
        if a[0].lower() == "t":
            qbank3.append((q,True))
        else:
            qbank3.append((q,False))
    elif len(qbank4) <10 :
        if a[0].lower() == "t":
            qbank4.append((q,True))
        else:
            qbank4.append((q,False))
for line in qbank1:
    print(line)

('The universe is 13.8 billion years old.', True)
('The constellation Virgo is the hottest place in the universe.', True)
('Jupiter is the largest planet in our solar system.', True)
('Apollo 17 was the last NASA manned mission space filght to the moon.', True)
('There is only one moon in our solar system.', False)
('The shortest space flight was 25 minutes.', False)
('Light cannot escape from a black hole.', True)
('There are 9 planets in the solar system.', False)
('Red supergiant stars are the largest type of stars in the universe.', True)
('Elliptical galaxies are not the most common galaxies in the universe.', False)


In [7]:
#detect events
def event_function(R,Fs):    
    window_size = Fs*1
    start_point = 0
    event_list = []
    var_list = []
    var_start = 0
    
    while True:
        if var_start+len(R)/Fs/10 >= len(R):
            break
        win = R[int(var_start):int(var_start+Fs/2)]
        var_list.append(np.var(win))
        var_start+=len(R)/Fs/3

    while True:
        if start_point+window_size<=len(R):
            window = R[int(start_point):int((start_point+window_size/2))]
            window_var = np.var(window)
            if window_var >= 0.052:
                event_list.append(start_point)
                start_point+=Fs*1.25
            else:
                start_point += len(R)/Fs/3
            if start_point+len(R)/Fs/3 >=len(R):
                break
        else:
            break
#     print(len(event_list))
#     print(event_list)
#     plt.figure()
#     plt.plot(var_list)
    return event_list
#detect direction
def direction_function(event_list,Fs):    
    direction_list = []
    for event in event_list:
        event_period = R[int(event):int(event+Fs/2)]
        max_pos=np.argwhere(R==max(event_period))
        min_pos=np.argwhere(R==min(event_period))
        if max_pos <min_pos:
            direction_list.append("L")
        else:
            direction_list.append("R")
#     print(direction_list)
#     print(np.mean(var_list))
    return direction_list
#machine learning
def point_interval(event_list,R,Fs):
    result_list = []
    for ele in event_list:
        result_list.append(R[int(ele):int(ele+Fs/2)])
    return result_list   
def randomforest(event_list):    
    features = np.array(event_list).astype(float)
    #labels = [1,1,1,1,1,0,0,0,0,0]
    labels = [1,0,1,0,1,0,1,0,1,0]
    labels = np.array(labels)
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    rf.fit(features,labels)
    return rf



In [8]:
def clean_function(R):
    Fs = 10000;
    T = np.linspace(1, len(R), len(R))/Fs
    nffte = len(T)
    X = np.fft.fft(R,nffte)
    X = np.fft.fftshift(X)

    f = np.linspace(-nffte/2,nffte/2-1,len(X))*Fs/nffte


    X[abs(f)>15]=0

    R1 = np.fft.ifftshift(X)
    R = np.fft.ifft(R1)
    return R

def normalize(R):
    #transfer to normalize graph
    max_r = max(R)
    min_r = min(R)
    if abs(min_r) > max_r:
        max_r = abs(min_r)
    for index in range(len(R)):
        R[index] /= max_r
    plt.figure()
    plt.plot(R)
    return R

In [40]:
from scipy.io import wavfile
#[Fs,R]=wavfile.read('new calibrate.wav')
#[Fs,R]=wavfile.read('regular calibration 2 end.wav')

[Fs,R]=wavfile.read('calibration1.wav')
Rsequence = [1,1,1,1,1,0,0,0,0,0];
Rsequence1 = [0,0,0,0,0,1,1,1,1,1];
#[Fs,R]=wavfile.read('calibration2.wav')
#[Fs,R]=wavfile.read('calibration3.wav')
#[Fs,R]=wavfile.read('calibration4.wav')


R = clean_function(R)#R is calibration


threshold = np.linspace(0.2,0.5,100)


R = clean_function(R)#R is calibration 


maxR = max(abs(R))
T = np.linspace(1, len(R), len(R))/Fs;
tsave = [];

for d in range(len(threshold)):

    TWindow = 0.05;    # observation interval
    threshold_level = threshold[d];

    TMax = max(T);  # maximum time recorded
    dT = T[1]-T[0]; # dT
    NWindow = TWindow/dT;   # number of points in Window
    delay_time = TWindow;   # wait the "recording interval" time
    Ra = R/maxR;

    r = [];
    time = [];
    RT = [];
    reset = 0;
    

    a = round(TMax/TWindow)

    for k in range(int(a)):



        window_index_beginning = 1+NWindow*(k);
        window_index_end = NWindow+NWindow*(k);
        window_index_range = np.linspace(window_index_beginning,window_index_end,window_index_end-window_index_beginning+1)


        blinkstest = max(abs(Ra[int(window_index_beginning):int(window_index_end)]));


        average_signal = sum(Ra[int(window_index_beginning):int(window_index_end)])/len(window_index_range);


        if reset % 30 == 0:
            reset = 0;


        if reset == 0:
            if blinkstest > 1.3:
                r = np.append(r,5);
                time = np.append(time,[TWindow*(k+1)]);
                reset = 1;
            elif average_signal>threshold_level:
                r = np.append(r,1);
                time = np.append(time,[TWindow*(k+1)]);
                reset = 1;
            elif average_signal<-threshold_level:
                r = np.append(r,0);
                time = np.append(time,[TWindow*(k+1)]);
                reset = 1;

        else:
            reset = reset + 1; 



    RT = np.stack((r, time))

    RT = RT.transpose()

    
    r = r[0:len(Rsequence)];
 
    
    acca = SequenceMatcher(None, r, Rsequence);
    acca = acca.ratio()
    
    acca1 = SequenceMatcher(None,r,Rsequence1)
    acca1 = acca1.ratio()
    
    #print(RT)
    
    if acca == 1:
        tsave = np.append(tsave,threshold_level)
        sign = 1;
    elif acca1 == 1:
        tsave = np.append(tsave,threshold_level)
        sign = -1;



fourfive = round(0.8*len(tsave))
trem = tsave
threshold_level = tsave[int(fourfive)]



In [41]:
threshold_level

0.3666666666666667

In [48]:
#METHOD 1 qbank 1

from scipy.io import wavfile
[Fs,Y]=wavfile.read('quiz1.wav')

Y = sign*clean_function(Y)/maxR

T = np.linspace(1, len(Y), len(Y))/Fs

TMax = max(T);  # maximum time recorded
TWindow = 0.05;    # observation interval
#threshold_level = 0.36;
dT = T[1]-T[0]; # dT
NWindow = int(round(TWindow/dT));   # number of points in Window

import time
r = [];
rime = [];
RTtrue = [];
reset = 0;
reset_2 = 0;
reset_3 = 0;

tr = [];
#import random


p = 0
# read = [None] * len(qbank1)
read = qbank1 #change this line in order to read different qbank (e.g.'read = qbank3' to read qbank3)


#random.shuffle(qbank)
correct = 0
s = 0

print("Left = True, Right = False\n")
q,a = read[p]
print(p+1,q)

#Run time: int(round(50/TWindow))
c = round(len(Y)/Fs)-1

#1, 8.19

for i in range(0,int(c/TWindow)):
    #time.sleep(0.05)
    if i>=155:

        

        if reset_2 != 0:
            if reset_2 % 50 == 0:
                p_1 = p % 10;
                #print new question
                q,a = read[p_1]
                #print ('\n')
                print(p+1,q)
                reset_2 = 0;

            elif reset_2 == 20:
                print('2 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 40:
                print('1 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 60:
                print('4 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 80:
                print('3 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 100:
                print('2 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 120:
                print('1 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;

            else:
                reset_2 = reset_2 + 1;

        elif reset_2 == 0:
            if reset != 0:
                if reset % 50 == 0:
                    #print new question
                    p_1 = p % 10;
                    q,a = read[p_1]
                    #print('\n')
                    print(p+1,q)
                    reset = 0;

                elif reset == 20:
                    print('2 second\r',end='',flush=True)
                    reset = reset + 1;
                elif reset == 40:
                    print('1 second\r',end='',flush=True)
                    reset = reset + 1;
                else:
                    reset = reset + 1;


        if reset ==0:
            if reset_2 == 0:

                r = [];
                rime = [];


                window_index_beginning = int(1+NWindow*(i)); # test if needs 1 or not
                window_index_end = int(NWindow+NWindow*(i));


                blinkstest = max(abs(Y[window_index_beginning:window_index_end]));


                average_signal = sum(Y[window_index_beginning:window_index_end])/(window_index_end-window_index_beginning+1)





                if reset_3 == 0:

                    if blinkstest > 1.3:
                        print('You skipped')
                        print(q,a,'\n')
                        #r = np.append(r,'Blink');
                        r = np.append(r,5)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        #reset = 1
                        reset_2 = 1;
                        reset_3 = 1;
                        p += 1
                        s += 1

                        #q,a = qbank[p]

                if reset_2 == 0:
                    if average_signal>threshold_level:

                        #r = np.append(r,'Left');
                        r = np.append(r,1)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        reset = 1;
                        usera= True
                        #usera = bool(usera)
                        # --- set user = none, while it's none, have the spiker box scan for signals; set a while loop to 
                        if usera == bool(a):
                            print("You said True, that is correct")
                            correct+= 1
                            #print(q,a)

                        else:
                            print("You said True, that is Incorrect")
                            #print(q,a)

                        print(q,a,'\n')
                        p += 1

                        #q,a = qbank[p]    
                        #print(p+1,q)
                        reset_3 = 0

                    elif average_signal<-threshold_level:

                        #r = np.append(r,'Right');
                        r = np.append(r,0)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        reset = 1;
                        usera= False
                        #usera = bool(usera)
                        # --- set user = none, while it's none, have the spiker box scan for signals; set a while loop to 
                        if usera == bool(a):
                            print("You said False, that is correct")
                            #print(q,a)
                            correct+= 1  

                        else:
                            print("You said False, that is incorrect")
                            #print(q,a)
                            #d.pop(a[0])
                        print(q,a,'\n')
                        p += 1
                        #q,a = qbank[p]
                        #print(p+1,q)
                        reset_3 = 0
                    #else:
                        #reset = reset + 1



                RT = np.stack((r, rime))

                RT = RT.transpose()  
                
                tr = np.append(tr,r)

                RTtrue = np.append(RTtrue,RT)
                
        

q1s = [1,0,1,0,0,1,1,1,1,0]  
tr = tr[0:len(tr)-1]

accur1 = SequenceMatcher(None,tr,q1s)

accur1 = accur1.ratio()
print('Accuracy =',accur1,', discount last reading as that is an error when turning off recording.')
    
print(RTtrue)
    
print("Time is up!", "Your Score:", correct, "Out of", p-s, ", Skipped:", s)

Left = True, Right = False

1 The universe is 13.8 billion years old.
You said True, that is correct
The universe is 13.8 billion years old. True 

2 The constellation Virgo is the hottest place in the universe.
You said False, that is incorrect
The constellation Virgo is the hottest place in the universe. True 

3 Jupiter is the largest planet in our solar system.
You said True, that is correct
Jupiter is the largest planet in our solar system. True 

4 Apollo 17 was the last NASA manned mission space filght to the moon.
You said False, that is incorrect
Apollo 17 was the last NASA manned mission space filght to the moon. True 

5 There is only one moon in our solar system.
You said False, that is correct
There is only one moon in our solar system. False 

6 The shortest space flight was 25 minutes.
You said True, that is Incorrect
The shortest space flight was 25 minutes. False 

7 Light cannot escape from a black hole.
You said False, that is incorrect
Light cannot escape from a bla

In [43]:
#METHOD 1 qbank2

from scipy.io import wavfile
[Fs,Y]=wavfile.read('quiz2.wav')

Y = sign*clean_function(Y)/maxR

T = np.linspace(1, len(Y), len(Y))/Fs

TMax = max(T);  # maximum time recorded
TWindow = 0.05;    # observation interval
#threshold_level = 0.33;
dT = T[1]-T[0]; # dT
NWindow = int(round(TWindow/dT));   # number of points in Window

import time
r = [];
rime = [];
RTtrue = [];
reset = 0;
reset_2 = 0;
reset_3 = 0;

tr = [];


#import random


p = 0
# read = [None] * len(qbank1)
read = qbank2 #change this line in order to read different qbank (e.g.'read = qbank3' to read qbank3)


#random.shuffle(qbank)
correct = 0
s = 0

print("Left = True, Right = False\n")
q,a = read[p]
print(p+1,q)

#Run time: int(round(50/TWindow))
c = round(len(Y)/Fs)-1



#1, 6.29s


for i in range(0,int(c/TWindow)):
    #time.sleep(0.05)
    if i>=115:

        

        if reset_2 != 0:
            if reset_2 % 50 == 0:
                p_1 = p % 10;
                #print new question
                q,a = read[p_1]
                #print ('\n')
                print(p+1,q)
                reset_2 = 0;

            elif reset_2 == 20:
                print('2 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 40:
                print('1 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 60:
                print('4 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 80:
                print('3 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 100:
                print('2 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 120:
                print('1 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;

            else:
                reset_2 = reset_2 + 1;

        elif reset_2 == 0:
            if reset != 0:
                if reset % 50 == 0:
                    #print new question
                    p_1 = p % 10;
                    q,a = read[p_1]
                    #print('\n')
                    print(p+1,q)
                    reset = 0;

                elif reset == 20:
                    print('2 second\r',end='',flush=True)
                    reset = reset + 1;
                elif reset == 40:
                    print('1 second\r',end='',flush=True)
                    reset = reset + 1;
                else:
                    reset = reset + 1;


        if reset ==0:
            if reset_2 == 0:

                r = [];
                rime = [];


                window_index_beginning = int(1+NWindow*(i)); # test if needs 1 or not
                window_index_end = int(NWindow+NWindow*(i));


                blinkstest = max(abs(Y[window_index_beginning:window_index_end]));


                average_signal = sum(Y[window_index_beginning:window_index_end])/(window_index_end-window_index_beginning+1)





                if reset_3 == 0:

                    if blinkstest > 1.3:
                        print('You skipped')
                        print(q,a,'\n')
                        #r = np.append(r,'Blink');
                        r = np.append(r,5)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        #reset = 1
                        reset_2 = 1;
                        reset_3 = 1;
                        p += 1
                        s += 1

                        #q,a = qbank[p]

                if reset_2 == 0:
                    if average_signal>threshold_level:

                        #r = np.append(r,'Left');
                        r = np.append(r,1)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        reset = 1;
                        usera= True
                        #usera = bool(usera)
                        # --- set user = none, while it's none, have the spiker box scan for signals; set a while loop to 
                        if usera == bool(a):
                            print("You said True, that is correct")
                            correct+= 1
                            #print(q,a)

                        else:
                            print("You said True, that is Incorrect")
                            #print(q,a)

                        print(q,a,'\n')
                        p += 1

                        #q,a = qbank[p]    
                        #print(p+1,q)
                        reset_3 = 0

                    elif average_signal<-threshold_level:

                        #r = np.append(r,'Right');
                        r = np.append(r,0)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        reset = 1;
                        usera= False
                        #usera = bool(usera)
                        # --- set user = none, while it's none, have the spiker box scan for signals; set a while loop to 
                        if usera == bool(a):
                            print("You said False, that is correct")
                            #print(q,a)
                            correct+= 1  

                        else:
                            print("You said False, that is incorrect")
                            #print(q,a)
                            #d.pop(a[0])
                        print(q,a,'\n')
                        p += 1
                        #q,a = qbank[p]
                        #print(p+1,q)
                        reset_3 = 0
                    #else:
                        #reset = reset + 1



                RT = np.stack((r, rime))

                RT = RT.transpose() 
                
                tr = np.append(tr,r)


                RTtrue = np.append(RTtrue,RT)
                
        

q2s = [1,1,1,1,0,1,1,0,0,1]   

tr = tr[0:len(tr)-2]

accur2 = SequenceMatcher(None,tr,q2s)

accur2 = accur2.ratio()
print('Accuracy =',accur2,', discount last 2 readings as that is an error when turning off recording.')
    
print(RTtrue)
    
print("Time is up!", "Your Score:", correct, "Out of", p-s, ", Skipped:", s)

Left = True, Right = False

1 An ant can fit 100 times its own weight.
You said True, that is Incorrect
An ant can fit 100 times its own weight. False 

2 8128 is a perfect number.
You said True, that is correct
8128 is a perfect number. True 

3 There are 24 time zones in the world.
You said True, that is correct
There are 24 time zones in the world. True 

4 secondAB postitive is the rarest blood type in humans.
You said True, that is Incorrect
AB postitive is the rarest blood type in humans. False 

5 An octopus has only one heart.
You said True, that is Incorrect
An octopus has only one heart. False 

6 Hydrogen is the first element on the periodic table.
You said False, that is incorrect
Hydrogen is the first element on the periodic table. True 

7 There are seven colors in a rainbow.
You said True, that is correct
There are seven colors in a rainbow. True 

8 Dolphins cannot smell.
You said False, that is incorrect
Dolphins cannot smell. True 

9 Liver is the largest organ of hum

In [46]:
#METHOD 1 qbank3

from scipy.io import wavfile
[Fs,Y]=wavfile.read('quiz3.wav')

Y = sign*clean_function(Y)/maxR

T = np.linspace(1, len(Y), len(Y))/Fs

TMax = max(T);  # maximum time recorded
TWindow = 0.05;    # observation interval
#threshold_level = 0.36;
dT = T[1]-T[0]; # dT
NWindow = int(round(TWindow/dT));   # number of points in Window

import time
r = [];
rime = [];
RTtrue = [];
reset = 0;
reset_2 = 0;
reset_3 = 0;

tr = [];


#import random


p = 0
# read = [None] * len(qbank1)
read = qbank3 #change this line in order to read different qbank (e.g.'read = qbank3' to read qbank3)


#random.shuffle(qbank)
correct = 0
s = 0

print("Left = True, Right = False\n")
q,a = read[p]
print(p+1,q)

#Run time: int(round(50/TWindow))
c = round(len(Y)/Fs)-1


#1, 4.72

for i in range(0,int(c/TWindow)):
    #time.sleep(0.05)
    if i>=80:

        

        if reset_2 != 0:
            if reset_2 % 50 == 0:
                p_1 = p % 10;
                #print new question
                q,a = read[p_1]
                #print ('\n')
                print(p+1,q)
                reset_2 = 0;

            elif reset_2 == 20:
                print('2 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 40:
                print('1 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 60:
                print('4 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 80:
                print('3 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 100:
                print('2 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 120:
                print('1 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;

            else:
                reset_2 = reset_2 + 1;

        elif reset_2 == 0:
            if reset != 0:
                if reset % 50 == 0:
                    #print new question
                    p_1 = p % 10;
                    q,a = read[p_1]
                    #print('\n')
                    print(p+1,q)
                    reset = 0;

                elif reset == 20:
                    print('2 second\r',end='',flush=True)
                    reset = reset + 1;
                elif reset == 40:
                    print('1 second\r',end='',flush=True)
                    reset = reset + 1;
                else:
                    reset = reset + 1;


        if reset ==0:
            if reset_2 == 0:

                r = [];
                rime = [];


                window_index_beginning = int(1+NWindow*(i)); # test if needs 1 or not
                window_index_end = int(NWindow+NWindow*(i));


                blinkstest = max(abs(Y[window_index_beginning:window_index_end]));


                average_signal = sum(Y[window_index_beginning:window_index_end])/(window_index_end-window_index_beginning+1)





                if reset_3 == 0:

                    if blinkstest > 1.3:
                        print('You skipped')
                        print(q,a,'\n')
                        #r = np.append(r,'Blink');
                        r = np.append(r,5)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        #reset = 1
                        reset_2 = 1;
                        reset_3 = 1;
                        p += 1
                        s += 1

                        #q,a = qbank[p]

                if reset_2 == 0:
                    if average_signal>threshold_level:

                        #r = np.append(r,'Left');
                        r = np.append(r,1)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        reset = 1;
                        usera= True
                        #usera = bool(usera)
                        # --- set user = none, while it's none, have the spiker box scan for signals; set a while loop to 
                        if usera == bool(a):
                            print("You said True, that is correct")
                            correct+= 1
                            #print(q,a)

                        else:
                            print("You said True, that is Incorrect")
                            #print(q,a)

                        print(q,a,'\n')
                        p += 1

                        #q,a = qbank[p]    
                        #print(p+1,q)
                        reset_3 = 0

                    elif average_signal<-threshold_level:

                        #r = np.append(r,'Right');
                        r = np.append(r,0)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        reset = 1;
                        usera= False
                        #usera = bool(usera)
                        # --- set user = none, while it's none, have the spiker box scan for signals; set a while loop to 
                        if usera == bool(a):
                            print("You said False, that is correct")
                            #print(q,a)
                            correct+= 1  

                        else:
                            print("You said False, that is incorrect")
                            #print(q,a)
                            #d.pop(a[0])
                        print(q,a,'\n')
                        p += 1
                        #q,a = qbank[p]
                        #print(p+1,q)
                        reset_3 = 0
                    #else:
                        #reset = reset + 1



                RT = np.stack((r, rime))

                RT = RT.transpose()  

                tr = np.append(tr,r)
                
                RTtrue = np.append(RTtrue,RT)
                
        

q3s = [1,1,0,1,0,5,1,0,1,1]   

accur3 = SequenceMatcher(None,tr,q3s)

accur3 = accur3.ratio()
print('Accuracy =', accur3)
    
print(RTtrue)
    
print("Time is up!", "Your Score:", correct, "Out of", p-s, ", Skipped:", s)

Left = True, Right = False

1 The Titanic sank in 1913.
You said True, that is Incorrect
The Titanic sank in 1913. False 

2 World War II ended in 1945.
You said True, that is correct
World War II ended in 1945. True 

3 There have been 50 U.S. Presidents.
You said False, that is correct
There have been 50 U.S. Presidents. False 

4 Queen Elizabeth II was born in 1926.
You said True, that is correct
Queen Elizabeth II was born in 1926. True 

5 Saint Patrick's Day was originally associated with color red.
You said False, that is correct
Saint Patrick's Day was originally associated with color red. False 

6 Zimbabwe was known as Rhodesia from 1964 to 1980.
You skipped
Zimbabwe was known as Rhodesia from 1964 to 1980. True 

7 The Cold war officially ended in 1979.
You said True, that is Incorrect
The Cold war officially ended in 1979. False 

8 Christopher Columbus discovered the New World in 1592.
You said False, that is correct
Christopher Columbus discovered the New World in 1592. F

In [50]:
#METHOD 1 qbank 4

from scipy.io import wavfile
[Fs,Y]=wavfile.read('quiz4.wav')

Y = sign*clean_function(Y)/maxR

T = np.linspace(1, len(Y), len(Y))/Fs

TMax = max(T);  # maximum time recorded
TWindow = 0.05;    # observation interval
#threshold_level = 0.36;
dT = T[1]-T[0]; # dT
NWindow = int(round(TWindow/dT));   # number of points in Window

import time
r = [];
rime = [];
RTtrue = [];
reset = 0;
reset_2 = 0;
reset_3 = 0;


tr = [];

#import random


p = 0
# read = [None] * len(qbank1)
read = qbank4 #change this line in order to read different qbank (e.g.'read = qbank3' to read qbank3)


#random.shuffle(qbank)
correct = 0
s = 0

print("Left = True, Right = False\n")
q,a = read[p]
print(p+1,q)

#Run time: int(round(50/TWindow))
c = round(len(Y)/Fs)-1


#1, 6.91


for i in range(0,int(c/TWindow)):
    #time.sleep(0.05)
    if i>=125:

        

        if reset_2 != 0:
            if reset_2 % 50 == 0:
                p_1 = p % 10;
                #print new question
                q,a = read[p_1]
                #print ('\n')
                print(p+1,q)
                reset_2 = 0;

            elif reset_2 == 20:
                print('2 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 40:
                print('1 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 60:
                print('4 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 80:
                print('3 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 100:
                print('2 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;
            elif reset_2 == 120:
                print('1 second\r',end='',flush=True)
                reset_2 = reset_2 + 1;

            else:
                reset_2 = reset_2 + 1;

        elif reset_2 == 0:
            if reset != 0:
                if reset % 50 == 0:
                    #print new question
                    p_1 = p % 10;
                    q,a = read[p_1]
                    #print('\n')
                    print(p+1,q)
                    reset = 0;

                elif reset == 20:
                    print('2 second\r',end='',flush=True)
                    reset = reset + 1;
                elif reset == 40:
                    print('1 second\r',end='',flush=True)
                    reset = reset + 1;
                else:
                    reset = reset + 1;


        if reset ==0:
            if reset_2 == 0:

                r = [];
                rime = [];


                window_index_beginning = int(1+NWindow*(i)); # test if needs 1 or not
                window_index_end = int(NWindow+NWindow*(i));


                blinkstest = max(abs(Y[window_index_beginning:window_index_end]));


                average_signal = sum(Y[window_index_beginning:window_index_end])/(window_index_end-window_index_beginning+1)





                if reset_3 == 0:

                    if blinkstest > 1.3:
                        print('You skipped')
                        print(q,a,'\n')
                        #r = np.append(r,'Blink');
                        r = np.append(r,5)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        #reset = 1
                        reset_2 = 1;
                        reset_3 = 1;
                        p += 1
                        s += 1

                        #q,a = qbank[p]

                if reset_2 == 0:
                    if average_signal>threshold_level:

                        #r = np.append(r,'Left');
                        r = np.append(r,1)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        reset = 1;
                        usera= True
                        #usera = bool(usera)
                        # --- set user = none, while it's none, have the spiker box scan for signals; set a while loop to 
                        if usera == bool(a):
                            print("You said True, that is correct")
                            correct+= 1
                            #print(q,a)

                        else:
                            print("You said True, that is Incorrect")
                            #print(q,a)

                        print(q,a,'\n')
                        p += 1

                        #q,a = qbank[p]    
                        #print(p+1,q)
                        reset_3 = 0

                    elif average_signal<-threshold_level:

                        #r = np.append(r,'Right');
                        r = np.append(r,0)
                        rime = np.append(rime,[TWindow*(i+1)]);
                        reset = 1;
                        usera= False
                        #usera = bool(usera)
                        # --- set user = none, while it's none, have the spiker box scan for signals; set a while loop to 
                        if usera == bool(a):
                            print("You said False, that is correct")
                            #print(q,a)
                            correct+= 1  

                        else:
                            print("You said False, that is incorrect")
                            #print(q,a)
                            #d.pop(a[0])
                        print(q,a,'\n')
                        p += 1
                        #q,a = qbank[p]
                        #print(p+1,q)
                        reset_3 = 0
                    #else:
                        #reset = reset + 1



                RT = np.stack((r, rime))

                RT = RT.transpose() 
                
                tr = np.append(tr,r)


                RTtrue = np.append(RTtrue,RT)
                
        
q4s = [1,1,1,0,5,1,5,0,1,0]  



accur4 = SequenceMatcher(None,tr,q4s)

accur4 = accur4.ratio()
print('Accuracy =',accur4)
    
print(RTtrue)
    
print("Time is up!", "Your Score:", correct, "Out of", p-s, ", Skipped:", s)

Left = True, Right = False

1 HP and Microsoft were all started in a Garage.
You said True, that is correct
HP and Microsoft were all started in a Garage. True 

2 Charles Babbage invented the first mechanical computer.
You said False, that is incorrect
Charles Babbage invented the first mechanical computer. True 

3 GUI stands for Graphical user interface in computer science.
You said False, that is incorrect
GUI stands for Graphical user interface in computer science. True 

4 Terabyte is the largest unit in storage.
You said True, that is Incorrect
Terabyte is the largest unit in storage. False 

5 Apple was the first publicly traded company to reach a 1 trillion dollar market cap.
You said False, that is incorrect
Apple was the first publicly traded company to reach a 1 trillion dollar market cap. True 

6 ROM stands for read only memory in computer memory.
You said True, that is correct
ROM stands for read only memory in computer memory. True 

7 SRGB has a wider range of colors t

In [116]:
#METHOD 2 MACHINE LEARNING


import random


p = 0
read = qbank2 #change this line in order to read different qbank (e.g.'read = qbank3' to read qbank3)


#random.shuffle(qbank)
correct = 0
s = 0

print("Left = True, Right = False\n")
q,a = read[p]
# print(p+1,q)

#Run time: int(round(50/TWindow))
c = round(len(Y)/Fs)-1



Y = Y[6*Fs:];

            
r = [];
rime = [];



average_signal = sum(Y)/(len(Y))

event_point = event_function(Y,10000)
events = point_interval(event_point,Y,10000)
user_answer = np.array(events).astype(float)


 
correct = 0
answer_list = []
#print(len(user_answer))
for num in range(0,len(user_answer)):
    #print(user_result)
    p_1 = p % 10
    
    q,a = read[p_1]
    print(p+1,q)
    user_result = rf.predict([user_answer[num]])

    if user_result > 0.4:
        answer_list.append(True)

        if a == True:
            
            r.append('correct')
            correct+=1
        else:
            r.append('not correct')
    else:
        answer_list.append(False)

        if a == False:
            r.append('correct')
            correct+=1
        else:
            r.append('not correct')
    print(r[len(r)-1],answer_list[len(answer_list)-1],a)
    print()
    p += 1

    
print(r)
print(answer_list)
print("Time is up!", "Your Score:", correct, "Out of", p-s, ", Skipped:", s)
print('First boolean is user input, second is true answer')







Left = True, Right = False

1 An ant can fit 100 times its own weight.
correct False False

2 8128 is a perfect number.
not correct False True

3 There are 24 time zones in the world.
not correct False True

4 AB postitive is the rarest blood type in humans.
correct False False

5 An octopus has only one heart.
correct False False

6 Hydrogen is the first element on the periodic table.
not correct False True

7 There are seven colors in a rainbow.
not correct False True

8 Dolphins cannot smell.
not correct False True

9 Liver is the largest organ of human body.
correct False False

10 Celsius and Fahrenheit are equal at -40 degrees.
not correct False True

11 An ant can fit 100 times its own weight.
correct False False

12 8128 is a perfect number.
not correct False True

13 There are 24 time zones in the world.
not correct False True

14 AB postitive is the rarest blood type in humans.
not correct True False

15 An octopus has only one heart.
not correct True False

16 Hydrogen is th